# $B_e$ equivalent linearized damping

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

from sklearn.metrics import r2_score

import copy
from rolldecay import database

from rolldecay.paper_writing import save_fig
from rolldecay.froude_scaling import froude_scale

import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import run, lambdify, significant_numbers

In [ ]:
db = database.get_db()

In [ ]:
limit_score = 0.99

df_linear = database.load(rolldecay_table_name='rolldecay_linear_b', limit_score=limit_score, 
                             exclude_table_name='rolldecay_exclude')
df_linear['B_2']=0

df_quadratic = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=limit_score, 
                             exclude_table_name='rolldecay_exclude')

df_cubic = database.load(rolldecay_table_name='rolldecay_cubic_b', limit_score=limit_score, 
                             exclude_table_name='rolldecay_exclude')

df_linear['ship_speed']*=1.852/3.6
df_quadratic['ship_speed']*=1.852/3.6

description = pd.read_sql_table('description', con=db.engine, index_col='id')
description.loc['ship_speed','unit']='m/s'
description.loc['VDES','unit']='m/s'
description.loc['Disp'] = {'description':'Ship discplacement','unit':'m3'}

df_linear['Disp']=df_linear['Volume']
df_quadratic['Disp']=df_quadratic['Volume']
df_cubic['Disp']=df_cubic['Volume']

skip=['omega0']  # This one is already model scale...
df_linear = froude_scale(data=df_linear, description=description, skip=skip)
df_quadratic = froude_scale(data=df_quadratic, description=description, skip=skip)
df_cubic = froude_scale(data=df_cubic, description=description, skip=skip)

g = 9.81
rho=1000
phi_a = df_linear['phi_start'].abs()
df_linear['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_linear, phi_a=phi_a)
df_linear['B_e_hat'] = run(function=lambdas.B_e_hat_lambda, inputs=df_linear, g=g, rho=rho)
df_linear['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_linear, g=g, rho=rho)
df_linear['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_linear, g=g, rho=rho)

phi_a = df_quadratic['phi_start'].abs()
df_quadratic['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_quadratic, phi_a=phi_a)
df_quadratic['B_e_hat'] = run(function=lambdas.B_e_hat_lambda, inputs=df_quadratic, g=g, rho=rho)
df_quadratic['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_quadratic, g=g, rho=rho)
df_quadratic['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_quadratic, g=g, rho=rho)

In [ ]:
df_compare = pd.merge(left=df_linear, right=df_quadratic, how='inner', 
                      left_index=True, right_index=True, suffixes=('','_quadratic'))

df_compare = pd.merge(left=df_compare, right=df_cubic, how='inner', 
                      left_index=True, right_index=True, suffixes=('','_cubic'))


In [ ]:
fig,ax = plt.subplots()
df_compare.plot(x='B_1', y='B_1_quadratic', style='.', ax=ax)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')
ax.set_aspect('equal', 'box')
ax.grid(True)
ax.set_xlabel('$B_1$ (linear)')
ax.set_ylabel('$B_1$ (quadratic)')

In [ ]:
fig,ax = plt.subplots()
df_compare.plot(x='B_1_quadratic', y='B_2_quadratic', style='.', ax=ax)

ax.grid(True)
ax.set_xlabel('$B_1$ (quadratic)')
ax.set_ylabel('$B_2$ (quadratic)')

In [ ]:
fig,ax = plt.subplots()
df_compare.plot(x='B_e', y='B_e_quadratic', style='.', ax=ax)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')
ax.set_aspect('equal', 'box')
ax.grid(True)
ax.set_xlabel('$B_e$ (linear)')
ax.set_ylabel('$B_e$ (quadratic)')

In [ ]:
lambdas.B_e_lambda_cubic

In [ ]:
def linearize(phi_a, df_linear, df_quadratic, df_cubic):
    df_linear=df_linear.copy()
    df_quadratic=df_quadratic.copy()
    df_cubic=df_cubic.copy()
    
    df_linear['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_linear, phi_a=phi_a)
    df_quadratic['B_e'] = run(function=lambdas.B_e_lambda, inputs=df_quadratic, phi_a=phi_a)
    df_cubic['B_e'] = run(function=lambdas.B_e_lambda_cubic, inputs=df_cubic, phi_a=phi_a)
    
    df_linear['B_e_hat'] = run(function=lambdas.B_e_hat_lambda, inputs=df_linear, g=g, rho=rho)
    df_quadratic['B_e_hat'] = run(function=lambdas.B_e_hat_lambda, inputs=df_quadratic, g=g, rho=rho)
    df_cubic['B_e_hat'] = run(function=lambdas.B_e_hat_lambda, inputs=df_cubic, g=g, rho=rho)
    
    df3 = df_compare = pd.merge(left=df_linear, right=df_quadratic, how='inner', 
                      left_index=True, right_index=True, suffixes=('','_quadratic'))
    
    df3 = pd.merge(left=df3, right=df_cubic, how='inner', 
                      left_index=True, right_index=True, suffixes=('','_cubic'))
    
    return df3
    

In [ ]:
g = 9.81
rho=1000
phi_as = np.deg2rad(np.linspace(0,6,30))

df1=df_linear.copy()
df2=df_quadratic.copy()

rms=[]
rms_cubic=[]
for phi_a in phi_as:

    df3 = linearize(phi_a=phi_a, df_linear=df_linear, df_quadratic=df_quadratic, df_cubic=df_cubic)
    
    df3['error']=df3['B_e_hat']-df3['B_e_hat_quadratic']
    df3['error_cubic']=df3['B_e_hat']-df3['B_e_hat_cubic']
    
    rms.append(np.sqrt(np.mean(df3['error']**2)))
    rms_cubic.append(np.sqrt(np.mean(df3['error_cubic']**2)))


In [ ]:
fig,ax=plt.subplots()
ax.plot(np.rad2deg(phi_as), rms, label='quadratic')
ax.plot(np.rad2deg(phi_as), rms_cubic, label='cubic')
ax.grid()
ax.legend()
ax.set_xlabel('$\phi_a$ [deg]')
ax.set_ylabel(r'$RMS\left(B_e(linear)-B_e(quadratic)\right)$');


In [ ]:
df_compare = linearize(phi_a=np.deg2rad(4.30), df_linear=df_linear, df_quadratic=df_quadratic,
                      df_cubic=df_cubic)

In [ ]:
fig,ax = plt.subplots()
x=df_compare['B_1_hat']
y=df_compare['B_e_hat_quadratic']
ax.plot(x, y,'.',label='quadratic')

x=df_compare['B_1_hat']
y=df_compare['B_e_hat_cubic']
ax.plot(x, y,'.',label='cubic')

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')
ax.set_aspect('equal', 'box')
ax.grid(True)
ax.set_xlabel('$\hat{B_1}$ (linear)')
ax.set_ylabel('$\hat{B_e}$')
ax.legend()

In [ ]:
r2_score(y_true=df_compare['B_e'], y_pred=df_compare['B_e_quadratic'])

In [ ]:
r2_score(y_true=df_compare['B_e'], y_pred=df_compare['B_e_cubic'])

In [ ]:
fig,ax=plt.subplots()
np.rad2deg(df_compare['phi_start_quadratic']).abs().hist(bins=30,ax=ax)

In [ ]:
fig,ax=plt.subplots()
np.rad2deg(df_compare['phi_stop_quadratic']).abs().hist(bins=30,ax=ax)

In [ ]:
np.rad2deg(df_compare['phi_start_quadratic'].abs() - df_compare['phi_stop_quadratic'].abs()).mean()

In [ ]:
df_linear['B_1'].isnull().any()

In [ ]:
df_cubic['B_1'].isnull().any()

In [ ]:
df_cubic['B_1'].isnull().sum()

In [ ]:
len(df_cubic)

In [ ]:
len(df_quadratic)

In [ ]:
len(df_linear)

In [ ]:
len(df_compare)